In [1]:
from fastembed.sparse.bm25 import Bm25
from fastembed import TextEmbedding
import pandas as pd
from docling.document_converter import DocumentConverter
from io import StringIO
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv(override=True)

os.chdir("..")

/home/giuseppe/projetos-pessoais/test-ai-engineer/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from src.data.preprocess import (
    extract_technic_specification,
    extract_html_from_pdf,
    tables_from_html,
    get_description_from_df,
)
import json

In [3]:
converter = DocumentConverter()


In [4]:
source = "../data/silver/FEC 2-2025 Scheda tecnica.pdf"

In [4]:
doc = converter.convert(source).document


In [5]:
# from IPython.core.display import  HTML
# display(HTML(doc.export_to_html()))

In [6]:
test = pd.read_html(StringIO(doc.export_to_html()))

In [7]:
df_technic_specification = pd.DataFrame()
for table in test:
    df_technic_specification = pd.concat([df_technic_specification, table], ignore_index=True)


In [8]:
df_technic_specification = df_technic_specification.T

In [9]:
df_technic_specification.columns = df_technic_specification.iloc[0]
df_technic_specification = df_technic_specification[1:]

In [10]:
df_technic_specification["Oggetto e descrizione del servizio"].values[0]

"Servizio di hosting, assistenza tecnica e manutenzione ordinaria, help desk e manutenzione evolutiva della piattaforma tecnologica Caronte. La stessa è stata sviluppata per la tenuta e la gestione del Registro Ufficiale degli Operatori Professionali (RUOP), di cui al Regolamento (UE) 2016/2031 e al D.lgs. 19/2021. Rappresenta lo strumento obbligatorio sia per gli operatori professionali che chiedono la registrazione al RUOP per lo svolgimento delle attività di vivaista, di commercio all'ingrosso, di importazione e di esportazione di vegetali, prodotti vegetali e altri oggetti, di autorizzazione all'uso del passaporto delle piante, di richiesta di emissione dei certificati fitosanitari per l'esportazione verso i Paesi terzi e sia per gli ispettori fitosanitari del Servizio fitosanitario regionale per la gestione dei controlli ufficiali obbligatori presso gli operatori professionali che svolgono l 'attività di vivaisti, il rilascio dei certificati fitosanitari di esportazione, l'adozion

In [11]:
df_technic_specification.columns = df_technic_specification.columns.str.lower()

In [15]:
columns_with_service = df_technic_specification.columns[df_technic_specification.columns.str.contains("servizio")]

In [25]:
description = ""
for text in df_technic_specification[columns_with_service].values:
    
    description += text[0] + "\n"

In [27]:
print(description)

Servizio di hosting, assistenza tecnica e manutenzione ordinaria, help desk e manutenzione evolutiva della piattaforma tecnologica Caronte. La stessa è stata sviluppata per la tenuta e la gestione del Registro Ufficiale degli Operatori Professionali (RUOP), di cui al Regolamento (UE) 2016/2031 e al D.lgs. 19/2021. Rappresenta lo strumento obbligatorio sia per gli operatori professionali che chiedono la registrazione al RUOP per lo svolgimento delle attività di vivaista, di commercio all'ingrosso, di importazione e di esportazione di vegetali, prodotti vegetali e altri oggetti, di autorizzazione all'uso del passaporto delle piante, di richiesta di emissione dei certificati fitosanitari per l'esportazione verso i Paesi terzi e sia per gli ispettori fitosanitari del Servizio fitosanitario regionale per la gestione dei controlli ufficiali obbligatori presso gli operatori professionali che svolgono l 'attività di vivaisti, il rilascio dei certificati fitosanitari di esportazione, l'adozione

In [ ]:
html = extract_html_from_pdf("./data/silver/scheda tecnica.pdf")

In [ ]:
openai = OpenAI()
response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
)
question = response.choices[0].message.content
print(question)

In [ ]:
def assistant_extractor(html_content: str, provider: str = "openai") -> str:
    """Extracts the description of a service from HTML content using an AI assistant."""
    system_prompt = (
        "You are an expert in extracting information from technical specifications. "
        "Your task is to extract the description of the service from the provided HTML content."
        "Don't invent information, just extract the relevant details."
        "Use the following HTML content to extract the description of the service:"
        ""
    )
    messages = [{"role": "system", "content": system_prompt}] + [
        {"role": "user", "content": html_content}
    ]

    if provider == "openai":
        openai = OpenAI()
        model = "gpt-4o"
    elif provider == "together":
        openai = OpenAI(
            api_key=os.getenv("TOGETHER_API_KEY"),
            base_url=os.getenv("TOGETHER_BASE_URL"),
        )
        model = "meta-llama/Llama-3.3-70B-Instruct-Turbo"
    else:
        raise ValueError("Provider must be 'openai' or 'together'.")

    response = openai.chat.completions.create(
        model=model,
        messages=messages,
    )
    answer = response.choices[0].message.content
    return answer


In [5]:
html = extract_html_from_pdf("./data/silver/Scheda tecnica gara Gestione Tremaglia.pdf")

In [7]:
from IPython.core.display import  HTML
display(HTML(html))

,"Dovrà inoltre essere garantita la presenza di una figura in possesso di esperienza documentabile nella catalogazione con il sistema SBN, di cui dovrà conoscere protocollo, procedure di allineamento e regole di catalogazione."
Sedi di esecuzione del servizio/fornitura,"Sede di Regione Lombardia (Palazzo Lombardia, in Piazza Città di Lombardia n. 1, Milano)."
Prodotti ed eventuali tempistiche,"- Continuità del servizio erogato all'utenza secondo quanto indicato nel paragrafo ' Oggetto e descrizione del servizio '; - verifica e aggiornamento del catalogo SBN/OPAC; - eventuale s carto del materiale ritenuto 'superato' ; - proposte per nuovi acquisti in linea con la natura del patrimonio bibliografico conservato in Biblioteca e per l'a ggiornamento della pagina web relativa ai 'Suggerimenti della preparazione di concorsi per bibliotecario' ( link); - produzione di una relazione sul lavoro svolto, con indicazione delle criticità riscontrate e delle potenziali aree di miglioramento, da inviare al dirigente responsabile della Biblioteca Tremaglia tramite PEC entro i 15 giorni successivi alla scadenza di ciascun trimestre. -"
Durata del contratto,Dalla sottoscrizione del Foglio Patti e Condizioni fino al 31/12/2027.
Importo massimo del servizio/fornitura e criteri con i quali è stato calcolato,"L'importo massimo stimato per il servizio è di 111.476 € IVA esclusa. Il costo della manodopera stimato per il servizio è di 100.328,40 €, pari al 90% dell'importo complessivo, calcolato sulla base della Tabella aggiornata (ottobre 2024) dal MINISTERO DEL LAVORO E DELLE POLITICHE SOCIALI - Direzione Generale dei Rapporti di Lavoro e delle Relazioni Industriali - Div. III - COSTO DEL LAVORO PER LE LAVORATRICI E I LAVORATORI DELLE COOPERATIVE DEL SETTORE SOCIO- SANITARIO ASSISTENZIALE-EDUCATIVO E DI INSERIMENTO LAVORATIVO Gli oneri per la sicurezza da interferenza sono pari a zero."
Modalità erogazione dei compensi,"Il pagamento verrà effettuato trimestralmente dietro presentazione di SAL e successiva fattura, sulla base di una relazione/rendicontazione attestante le attività realizzate e previa verifica da parte del DEC della regolare esecuzione del servizio."
Contenuti proposta,"- Presentazione delle idonee esperienze pregresse maturate dal l'operatore economico , comprese quelle relative all'utilizzo dei software di gestione della catalogazione SBN; - breve descrizione degli aspetti qualitativi del servizio offerto, nonché del personale richiesto e dei relativi titoli (max 10 cartelle, non allegare CV); - prezzo offerto."


In [7]:
service = assistant_extractor(html, provider="together")

In [9]:
print(service)

La descrizione del servizio è la seguente:

Il servizio di manutenzione delle reti radio di Regione Lombardia comprende la manutenzione preventiva e correttiva per il mantenimento nelle migliori condizioni di efficienza delle reti radio regionali, tra cui:

* La dorsale pluricanale regionale denominata "Alta Frequenza" (Anello nord e anello sud)
* Le reti radio ISO frequenziali ed apparati ricetrasmittenti terminali del servizio antincendio boschivo e di protezione civile
* I sistemi di comunicazione mobili installati sui CTM-R, CTM-P e Sala Operativa mobile (SOM), costituiti da ripetitori mobili DMR in configurazione master e satelliti, BST Tetra e connettività satellitare Tooway, Starlink e LTE 4G
* Gli apparati terminali, portatili, veicolari e fissi in tecnologia DMR, TETRA e aeronautici
* Il sistema di trouble ticketing
* La gestione dei servizi aggiuntivi (setup sistemi radio "nomadici", Drive test, Traslochi, Supporto in sede)

Il servizio dovrà essere effettuato su tutti gli ap